In [5]:
import pandas as pd
import geopandas as gp
import numpy as np

In [2]:
# layers: contours, building footprints, watershed boundary, roads, water channel

In [6]:
# set crs
crs = 2926

In [11]:
# set extents
extent_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Boundary/SP clipping area.shp"
extent = gp.read_file(extent_path)
extent = extent.to_crs(crs)

In [12]:
# set watershed boundary
watershed_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Processed/green_duwamish_watershed.shp"
watershed = gp.read_file(watershed_path)


In [13]:
# find intersection of watershed boundary and extent
watershed_extent_intersect = gp.overlay(watershed, extent, how='intersection')

Building processing:
-find all buidling centroids that fall within the watershed boundaries
-export the building footprints of those centroids

In [ ]:
# process building footprints to inside the extent
buildings_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Processed/buildings_projected_trim.shp"
buildings = gp.read_file(buildings_path)

In [ ]:
buildings['centroid_geom'] = buildings.centroid()
buildings.set_geometry('centroid_geom',inplace = True)
buildings_clip = gp.overlay(buildings, watershed_extent_intersect, how = "intersection")
buildings_clip.set_geometry('geom', inplace = True)
buildings_clip.to_file("/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Processed/buildings_clipped_WS_model.shp")

Contour processing:
-get 5' contours within the water channel boundary for greater definition
-20' contour resolution within the entire extent of the model area
-merge the 5' and 20' contour subsets
-export and create mesh in Rhino

In [ ]:
# process contours to inside the extent
# select all contours that fall within water channel boundary - processed in QGIS
water_channel_cont_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Processed/duwamish_river_contours.shp"
water_channel_contours = gp.read_file(water_channel_cont_path)

# select all contours that are 20' ft - processed in QGIS
contours_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Processed/clipped_20ft_contours.shp"
contours = gp.read_file(contours_path)

# clip to extent
contours_clip = gp.overlay(contours, extent, how = 'intersection')

# merge water channel contours and 20' contours
contours_merge = gp.merge(contours_clip, water_channel_contours)

Road processing
- clip to watershed extent ? clip to extent ?
- find average street width to backfill NaN
- buffer and export outlines

In [ ]:
streets_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Projected/seattle_streets_projected.shp"
streets = gp.read_file(streets_path)

streets['ARTCLASS'] = streets['ARTCLASS'].map(str)

## backfill data by average width for each functional class
# remove rows = 0 before finding average
streets_wid = streets[['ARTCLASS','SURFACEWID']]
streets_wid = streets_wid[streets_wid.SURFACEWID != 0]
streets_wid.SURFACEWID.value_counts()

## calculate average by arterial class and surface type
streets_wid_avg = streets_wid.groupby(['ARTCLASS']).mean()
streets_wid_avg.reset_index(inplace = True)
streets_wid_avg['ARTCLASS'] = streets_wid_avg['ARTCLASS'].map(str).str[0]


In [ ]:
# clip streets to extent
streets_clip = gp.overlay(streets, watershed_extent_intersect, how = "intersection")

In [ ]:
# backfill data in the CLIPPED AND PROJECTED FILE by average width for each functional class
# NOTE: arterial class == 9 has one feature with surface width == 0. there is no backfilled data for this road
streets_bfill = pd.merge(left = streets_clip, right = streets_wid_avg, how = 'left', left_on = 'ARTCLASS', right_on = 'ARTCLASS')
streets_bfill['width_upd'] = np.where(streets_bfill['SURFACEWID_x'] == 0, streets_bfill['SURFACEWID_y'], streets_bfill['SURFACEWID_x'])

In [ ]:
# buffer by the surface width
streets_bfill['geometry'] = streets_bfill.buffer(distance = streets_bfill['width_upd'])
columns = ['OBJECTID','ARTCLASS','UNITDESC','STNAME_ORD','ARTDESCRIP','SURFACEWID_x','width_upd','geometry']

# output buffers as polygons and as outline polylines
streets_bfill[columns].to_file("/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Processed/seattle_streets_buffer_WS_model.shp")
streets_bfill.dissolve().boundary.to_file("/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Processed/seattle_streets_buffer_outlines_WS_model.shp")